In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

import time

In [2]:
def load_images_and_labels(directory, target_size):
    images = []
    labels = []  

    for label, class_dir in enumerate(['fake', 'real']):
        class_dir_path = os.path.join(directory, class_dir)
        for file_name in os.listdir(class_dir_path):
            img_path = os.path.join(class_dir_path, file_name)
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(label)

    return np.array(images), np.array(labels)

In [3]:
start_time = time.time()

# Basis-Modell laden (ohne die obersten Schichten)
base_model = ResNet50(weights='imagenet', include_top=False)

# Eigene Klassifizierungsschichten hinzufügen
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)  # 1 Ausgang für binäre Klassifizierung

# Gesamtmodell konstruieren
model = Model(inputs=base_model.input, outputs=predictions)

# Alle Schichten im Basis-Modell einfrieren
for layer in base_model.layers:
    layer.trainable = False

# Modell kompilieren
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

dataset_dir = "../images/car"

img_height = 128
img_width = 128
batch_size = 1

images, labels = load_images_and_labels(dataset_dir, (img_height, img_width))

# Teile die Daten in Trainings- und Testdatensätze
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Initialisiere ImageDataGenerator
train_data_gen = ImageDataGenerator(rescale=1.0/255)
test_data_gen = ImageDataGenerator(rescale=1.0/255)

# Erstelle Trainings- und Testdatensätze mit dem ImageDataGenerator
train_ds = train_data_gen.flow(X_train, y_train, batch_size=batch_size)
val_ds = test_data_gen.flow(X_test, y_test, batch_size=batch_size, shuffle=False)

# Modell trainieren
model.fit(
    train_ds,
    steps_per_epoch=len(X_train) // batch_size,  # Verwende die Länge von X_train
    epochs=10,
    validation_data=val_ds,
    validation_steps=len(X_test) // batch_size  # Verwende die Länge von X_test
)

print("--- %s seconds ---" % (time.time() - start_time))


Epoch 1/10
11315/11315 [==============================] - 596s 53ms/step - loss: 0.4877 - accuracy: 0.7695 - val_loss: 0.4419 - val_accuracy: 0.8204
Epoch 2/10
11315/11315 [==============================] - 607s 54ms/step - loss: 0.4168 - accuracy: 0.8128 - val_loss: 0.4015 - val_accuracy: 0.8208
Epoch 3/10
11315/11315 [==============================] - ETA: 0s - loss: 0.3925 - accuracy: 0.8289